In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [14,14]


def show_image(image, description=''):
    plt.subplot()
    img = plt.imshow(image, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.title(description)
    plt.show()
    
def show_images(images, descriptions=[], layouts=''):
    if layouts == '':
        layouts = "{}1".format(len(images))
    if descriptions == []:
        descriptions = ['' for _ in range(len(images))]
    for i,img in enumerate(images):
        plt.subplot("{}{}".format(layouts, i+1))
        img = plt.imshow(img, cmap='gray', interpolation='nearest')
        plt.axis('off')
        plt.title(descriptions[i])
    plt.show()

## Difference of Gaussian

In [ ]:
ksize = 3
gaussian_filter = cv2.getGaussianKernel(ksize, -1)
gaussian_filter

In [ ]:
image = cv2.imread("quartergray.png", cv2.IMREAD_GRAYSCALE)/255

gaussian_filter3 = cv2.getGaussianKernel(3, -1)
smoothed3=cv2.sepFilter2D(image, -1, gaussian_filter3, gaussian_filter3.T)
gaussian_filter5 = cv2.getGaussianKernel(5, -1)
smoothed5=cv2.sepFilter2D(image, -1, gaussian_filter5, gaussian_filter5.T)

DoG = smoothed5-smoothed3
# cv2.NORM_MINMAX normalizes values to be in range [beta,alpha]
# Used for `imwrite` requiring `uint8` type ...
norm_DoG = cv2.normalize(DoG,None,alpha=255,beta=1,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8UC1)

edge_threshold = 0.1 * (np.max(DoG)-np.min(DoG))
edges = DoG > edge_threshold

images = [image,smoothed3,smoothed5,DoG,norm_DoG,edges]
descriptions = ['original', 'smoothed3', 'smoothed5', 'DoG', 'normDoG', 'edges']
show_images(images, descriptions, "32")

## Laplacian of Gaussian

In [ ]:
ksize = 3

image = cv2.imread("quartergray.png", cv2.IMREAD_GRAYSCALE)/255
gaussian_filter = cv2.getGaussianKernel(ksize, -1)
smoothed=cv2.sepFilter2D(image, -1, gaussian_filter, gaussian_filter.T)

laplacian_filter = np.array([ [0,1,0],[1,-4,1],[0,1,0] ])
LoG = cv2.filter2D(image,-1,laplacian_filter)

edge_threshold = 0.1 * (np.max(LoG)-np.min(LoG))
edges = LoG > edge_threshold

images = [image,smoothed,LoG,edges]
descriptions = ['original', 'smoothed', 'LoG', 'edges']
show_images(images, descriptions, "22")

## Gradient Filters

In [ ]:

def gradient(image, type='sobel'):
        # Creates a 2D convolutional filter (see https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html )
        # This is the same as performing for each pixel the finite difference calculation given in class
        if type == 'forwarddiff':
                filter_x = np.array(([0, 0, 0], [0, -1, 1], [0, 0, 0]))
                filter_y = filter_x.T
        if type == 'backwarddiff':
                filter_x = np.array(([0, 0, 0], [-1, 1, 0], [0, 0, 0]))
                filter_y = filter_x.T
        elif type == 'centraldiff':
                filter_x = np.array(([0, 0, 0], [-1, 0, 1], [0, 0, 0]))/2
                filter_y = filter_x.T
        elif type == 'smooth':
                filter_x = np.array(([0, 0, 0], [1, 2, 1], [0, 0, 0]))/4
                filter_y = filter_x.T
        else:
                filter_y = np.array(([-1, -2, -1], [0, 0, 0], [1, 2, 1]))
                filter_x = filter_y.T

        # filter the image
        grad_x = cv2.filter2D(image, -1, filter_x)
        grad_y = cv2.filter2D(image, -1, filter_y)

        return grad_x, grad_y


def polar(grad_x, grad_y):
    magnitude = np.sqrt(grad_x**2+grad_y**2)
    theta = np.arctan2(grad_y, grad_x)

    theta = theta/np.pi

    return magnitude.astype(np.float32), theta.astype(np.float32)
    

In [ ]:
image = cv2.imread("quartergray.png", cv2.IMREAD_GRAYSCALE)/255

filters = ['sobel','smooth','forwarddiff','backwarddiff','centraldiff']
for f in filters:
    (grad_x, grad_y) = gradient(image, f)
    descriptions = ["{}-{}".format(f,x) for x in ['gradx','grady','magnitude']]
    show_images([grad_x,grad_y,grad_x+grad_y], descriptions,'13')
